# Running Unity ML-Agents on SageMaker

This notebook contains instructions for setting up SageMaker for training ML-Agents environments.

![ml-agents](./images/image-banner.png)

Note: This tutorial supports only unity ml-agents v0.11.0.

## Setup
We'll begin with some necessary imports, and get an Amazon SageMaker session to help perform certain tasks, as well as an IAM role with the necessary permissions.

In [ ]:
# S3 prefix
prefix = '<bucket name>'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Building and registering the container

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=ml-agents-sagemaker

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
data_location = '<path of your env file>'

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

sess = sagemaker.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

image = '{}.dkr.ecr.{}.amazonaws.com/ml-agents-sagemaker:latest'.format(account, region)

estimator = Estimator(image,
                       role, 1, 'ml.m5.24xlarge',
                       output_path='<output file>',
                       sagemaker_session=sess)

estimator.fit({"train":data_location})

In [ ]:
!aws s3 ls <output file>

In [ ]:
!aws s3 cp <output file>/output/model.tar.gz .

In [ ]:
!tar -zxvf model.tar.gz

In [ ]:
!tensorboard --logdir=summaries

You can access to the tensorboad
"https://notebook_name.notebook.us-east-1.sagemaker.aws/proxy/6006/"